# 제 2 절  Auto Grad
Gradient 값을 자동으로 수행

## 01 Auto Grad
자동적으로 미분을 수행, Forward 작업의 내용을 기억 후, backward 작업시에는 기억 내용을 재생

## 02 Vaiable

### 1 Variable 클래스
<p>Tensor 생성된 데이터 주위를 Variable 클래스로 얇게 감싼다.</p></br>
<p>.data() : raw_tensor에 접근</p></br>
<p>.grad() : gradient 값이 누적저장

### 2 Function 클래스
<p>Variable 클래스와 상호관계를 맺으며 사슬모양 그래프를 만들고 계산과정의 History를 부호화 한다</p></br>
<p>.creator() : 자동으로 생성된 function을 참조 (사용자가 생성한 variable은 제외)</p></br>
<p>.backwardgrad() : Variable이 파생된 항목을 찾기 위한 함수
  1. 파생항목이 Scala : 인자를 특정할 필요가 없다
  2. 2개 이상을 갖는 tensor : .grad_output() 로 인자를 특정할 필요가 있다

In [1]:
import torch
from torch.autograd import Variable

In [2]:
# Variable(<data> , <grad>, <creator> )  # <creator>는 사용자 정의 함수
x = Variable(torch.ones(2,2), requires_grad=True)
x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [3]:
x.data


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [4]:
print(x.grad)

None


In [5]:
# 정의한 내용이 없어서 None
# Tensor 생성 후 임의 작업이 없었다
print(x.creator)

None


### 3 Variable 클래스의 계산

In [6]:
y = x + 2
y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [7]:
y.creator # 사용자가 작업한 과정이 저장됨이 확인

In [8]:
y * y

Variable containing:
 9  9
 9  9
[torch.FloatTensor of size 2x2]

In [9]:
z = y * y * 3
z

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]

In [10]:
out = z.mean()
out

Variable containing:
 27
[torch.FloatTensor of size 1]

### 4 Gradient
위 내용에 이어 backpropagation 을 계산 후, 편미분 값(Gradient)을 계산/ 출력

In [11]:
out.backward()

In [12]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



<p>Gradient 계산은 그래프(계산과정 전체)에 담겨진 모든 내부 buffer를 사용한다.</p></br>
만약, 특정 부분을 두번 backward 하려면, 첫 번쨰 pass 동안 retain_variables = True 를 설정해야 한다. 

In [13]:
x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [14]:
x = Variable(torch.ones(2,2), requires_grad=True)

In [15]:
y = 4 * x + 2
y

Variable containing:
 6  6
 6  6
[torch.FloatTensor of size 2x2]

In [16]:
# retain_variables=True 내부 buffer가 넘치는 것을 예방
y.backward(torch.ones(2,2), retain_variables=True)

In [17]:
# y.grad 로 하면 아무것도 안 나온다 (독립변수에 저장)
print(x.grad)

Variable containing:
 4  4
 4  4
[torch.FloatTensor of size 2x2]



In [18]:
z = y * y
z

Variable containing:
 36  36
 36  36
[torch.FloatTensor of size 2x2]

새로운 랜덤 gradient를 backprop 해보자

In [19]:
gradient = torch.randn(2,2)
gradient


 1.0932  1.9507
-1.0305 -0.7468
[torch.FloatTensor of size 2x2]

    y.backward()로 x 독립변수에 grad 영향을 준다
    tensorflow에서는 backward()연산도 전체적인 그래프가 완성 및 시행 중에만 가능하지만
    pytorch에서는 미완성 그래프에서도 다양한 미분을 통해서 알고리즘을 테스트 할 수 있다.
    (http://blog.aloni.org/posts/backprop-with-tensorflow/

In [20]:
# gradient 자체가 특정되지 않아서 x.grad의 결과는 계속 변한다
# y(종속변수)의 .backward() 메소드로 손쉽게 계산이 가능
y.backward(gradient, retain_variables = True)
x.grad

Variable containing:
  8.3726  11.8026
 -0.1220   1.0130
[torch.FloatTensor of size 2x2]

Autograd 의 활용

In [21]:
x = torch.randn(3)
x = Variable(x, requires_grad = True)
x

Variable containing:
 1.3570
-0.3653
 0.6265
[torch.FloatTensor of size 3]

In [22]:
y = x * 2
y

Variable containing:
 2.7140
-0.7306
 1.2530
[torch.FloatTensor of size 3]

In [23]:
# 난수값이 1000보다 큰 값이 나올 떄 까지 반복
while y.data.norm() < 1000:
    y = y * 2
print(y)

Variable containing:
 1389.5795
 -374.0446
  641.5152
[torch.FloatTensor of size 3]



In [24]:
gradients = torch.FloatTensor([ 0.1, 1.0, 0.0001])
y.backward(gradients)

In [25]:
x.grad

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]